In [1]:
# %load_ext autoreload
# %autoreload 2
%matplotlib inline

In [2]:
# import matplotlib
from matplotlib import pyplot as plt
from matplotlib import font_manager
import os

In [3]:
import os
import sys
sys.path.append('..')
import torch

import params
from lavse.data.loaders import get_loader
from lavse.model import model
from lavse.train.train import Trainer
from lavse.utils import file_utils, helper
from lavse.utils.logger import create_logger
from run import load_yaml_opts, parse_loader_name
from tqdm import tqdm
from lavse.data.collate_fns import Collate
from lavse.utils import layers
import json
from PIL import Image
import numpy as np
from lavse.train.evaluation import predict_loader
from matplotlib import font_manager as fm, rcParams
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import matplotlib
# matplotlib.rc('font', family='fonts-japanese-mincho')
# matplotlib.rc('font', family='Arial')
# matplotlib.rcParams['font.fantasy'] = 'Ubuntu'
from torchvision import transforms
collate = Collate()
from pathlib import Path

In [4]:
path = Path('/opt/jonatas/repos/lavse/logs_aaai2/f30k_precomp/adapt_t2i/glove-freeze-img_norm_softmax/')
opt = load_yaml_opts(path / 'options.yaml')
model_path = path / 'best_model.pkl'

In [5]:
opt['model']['txt_enc']['params']['glove_path'] = '/opt/jonatas/repos/lavse/.vocab_cache/glove_f30k_precomp.json.pkl'

In [6]:
opt['model']['txt_enc']

{'name': 'gru_glove',
 'params': {'embed_dim': 300,
  'use_bi_gru': True,
  'glove_path': '/opt/jonatas/repos/lavse/.vocab_cache/glove_f30k_precomp.json.pkl',
  'add_rand_embed': True},
 'pooling': 'none',
 'devices': ['cuda']}

In [7]:
ngpu = 1
loaders = []

if 'DATA_PATH' not in os.environ:
    data_path = '/opt/jonatas/datasets/lavse'
#     data_path = '../lavse'
#     data_path = opt.dataset.data_path
else:
    data_path = os.environ['DATA_PATH']


print(opt.dataset.lang)
opt.dataset.vocab_paths = ['../{}'.format(p) for p in opt.dataset.vocab_paths]
for data_name in opt.dataset.val.data:
    data_name, lang = parse_loader_name(data_name)
    print(data_name)
    loader = get_loader(
        data_split='test',
        data_path=data_path,
        data_name=data_name,
        loader_name=opt.dataset.loader_name,
        local_rank=0,
        lang=lang,
        text_repr=opt.dataset.text_repr,
        vocab_paths=opt.dataset.vocab_paths,
        ngpu=ngpu,
        **opt.dataset.val,
    )
    loaders.append(loader)

device = torch.device('cuda:0')

print(len(loaders))
loader = loaders[0]
tokenizers = loader.dataset.tokenizers
if type(tokenizers) != list:
    tokenizers = [tokenizers]

{}
f30k_precomp
Image div 1
1


In [8]:
# load model and options
model = model.LAVSE(**opt.model, tokenizers=tokenizers)#.to(device)
checkpoint = helper.restore_checkpoint(
    path=model_path,
    model=model,
)

model.set_devices_(
    txt_devices=[device],
    img_devices=[device],
    loss_device=device,
)
is_master = True
model.master = is_master # FIXME: Replace "if print" by built_in print


In [9]:
from lavse.model.similarity.measure import cosine_sim, l2norm

In [10]:
img_embs, cap_embs, cap_lens = predict_loader(model, loader, device)
print(img_embs.shape, cap_embs.shape)
# img_embs /= np.linalg.norm(img_embs, axis=1, keepdims=True)
# cap_embs /= np.linalg.norm(cap_embs, axis=1, keepdims=True)
# print(img_embs.shape, cap_embs.shape)
# cap_embs = l2norm(torch.tensor(cap_embs).float(), -1)
# img_embs = l2norm(torch.tensor(img_embs).mean(1).float(), -1)

(1000, 36, 1024) (5000, 77, 1024)


In [ ]:
def mean_pooling_with_len(x, lens):
    max_len = int(np.max(lens))
    mask = torch.arange(max_len).expand(len(lens), max_len) < torch.tensor(lens).long().unsqueeze(1)
    mask = mask.float().cpu()
    vector = x.sum(1) / mask.cpu().unsqueeze(2).sum(1)
    return vector

In [ ]:
with torch.no_grad():
    x = mean_pooling_with_len(torch.tensor(cap_embs).float().cpu(), cap_lens)

In [ ]:
cemb = l2norm(x, -1)
iemb = l2norm(torch.tensor(img_embs).mean(1).float(), -1)

In [ ]:
cemb.shape, iemb.shape

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(perplexity=40,)

In [ ]:
fit_data = np.concatenate([cemb[:500], iemb[:100]], 0)
print(fit_data.shape)
data = tsne.fit_transform(fit_data)

In [ ]:
import seaborn as sns

In [ ]:
import seaborn as sns; sns.set(style='white', )
colors = sns.color_palette()

In [ ]:
ci = np.arange(0, 600, 1) // 5
c = [colors[i % len(colors)] for i in ci]

In [ ]:
x = data[:,0]
y = data[:,1]

In [ ]:
print(x.shape)

In [ ]:
plt.scatter(x, y, c=c, alpha=0.3)

In [11]:
_ = model.similarity.similarity(
    torch.tensor(img_embs[:10]).float().cuda(), 
    torch.tensor(cap_embs[:50]).float().cuda(),
    cap_lens,
)

tensor([[[2.8286e-02, 5.7221e-02, 3.7450e-02,  ..., 2.0226e-02,
          9.1577e-02, 9.1944e-02],
         [7.6146e-03, 2.3379e-03, 6.0560e-03,  ..., 3.5624e-03,
          6.7657e-03, 4.6830e-03],
         [2.8352e-01, 1.3711e-02, 9.2421e-03,  ..., 2.9285e-03,
          2.1840e-04, 4.3831e-08],
         ...,
         [3.0719e-05, 3.6107e-02, 7.4170e-04,  ..., 2.7239e-05,
          6.0069e-05, 5.2305e-03],
         [8.3180e-03, 2.1354e-01, 4.7016e-02,  ..., 4.7950e-03,
          2.4076e-02, 3.8247e-03],
         [4.6009e-03, 1.7261e-02, 4.0310e-03,  ..., 1.2766e-02,
          4.8167e-04, 5.2097e-05]],

        [[3.3337e-04, 9.8366e-05, 2.1892e-04,  ..., 3.7477e-04,
          1.1196e-04, 4.5753e-04],
         [7.1099e-07, 9.4101e-04, 1.5661e-06,  ..., 4.9198e-04,
          2.2967e-06, 1.9031e-04],
         [1.2938e-03, 1.8615e-02, 1.0400e-02,  ..., 1.0214e-01,
          1.4984e-03, 9.1611e-02],
         ...,
         [4.2468e-04, 3.5940e-02, 5.4845e-04,  ..., 5.1939e-04,
          8.634

In [15]:
_[0].sum(0)

tensor([41.3943, 50.3701, 42.1236, 24.6865, 16.4362, 27.0679, 25.6940, 30.5770,
        12.3891, 24.7710, 18.2719, 26.3849, 40.6504, 25.8636, 36.9654, 36.6643,
        28.6649, 23.9550, 58.5975, 38.1512, 23.2272, 41.8049, 32.1977, 20.3253,
        17.3744, 20.6523, 22.2894, 28.1067, 24.7474, 30.7673, 27.4584, 13.4958,
        15.8274, 18.3241, 29.1415, 28.5817], device='cuda:0',
       grad_fn=<SumBackward2>)

In [ ]:
ids = np.load('/opt/jonatas/datasets/lavse/jap_precomp/test_ids.npy')
new_ids = np.zeros((len(ids)*5), dtype=np.int64
for i in range(len(ids)):
    new_ids[i*5:(i*5)+5] = ids[i]
    
loader.dataset.ids = new_ids
print(ids.min(), ids.max(), ids.mean())

jp_caps = np.loadtxt('/opt/jonatas/datasets/lavse/jap_precomp/test_caps.jp.txt', delimiter='@', dtype='unicode')

In [ ]:
# with open('/opt/jonatas/datasets/lavse/f30k/dataset_flickr30k.json', 'r') as fp:
#     flickr = json.load(fp)
#     f30k = {x['filename']: x for x in flickr['images']}
with open('/opt/jonatas/datasets/lavse/jap_precomp/yjcaptions26k/yjcaptions26k_clean.json', 'r') as fp:
    flickr = json.load(fp)

In [ ]:
new_info = {x['id']: x for x in flickr['images']}

flickr['images'][0]


In [ ]:
flickr['images'][0]

In [ ]:
img_embs, cap_embs, cap_lens = torch.tensor(img_embs), torch.tensor(cap_embs), torch.tensor(cap_lens)
sims = model.get_sim_matrix_shared(
        embed_a=img_embs, embed_b=cap_embs,
        lens=cap_lens, shared_size=128
    )
sims = layers.tensor_to_numpy(sims)
print(sims.shape)
new_sims = np.zeros((4885, 4885))
for i in range(977):
    new_sims[i*5:(i*5)+5] = sims[i]
sims = new_sims
print(sims.shape)

In [ ]:
print(sims.min(), sims.max(), sims.mean())
print(type(sims))

In [ ]:
len(flickr['images'])

# TEXT2IMAGE

In [ ]:
# text 2 img
lang = 'jp'
sims_t = sims.T
sims_t1k = sims_t[:, range(0, sims_t.shape[1], 5)]
print('sim1k', sims_t1k.shape)
inds = np.argsort(sims_t1k, axis=1)[:, ::-1]

ld = loader

cap_idx = 51

for cap_idx in np.random.choice(4885, 100, replace=False):
#     cap = ld.dataset.captions[cap_idx]
    cap = jp_caps[cap_idx]
    print(cap)

    n = 15
    if cap.endswith('.'):
        cap = cap[:-2]
    split_str = cap.split(' ')
    if len(split_str) > n:
        for j in range(n, len(split_str), n):
            split_str[j-1] += '\n   '
    cap = ' '.join(split_str)
    cap = '{}: {}'.format('Query', cap)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.gca().set_axis_off()
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.axis('off')
    plt.subplots_adjust(bottom=0, top=1, left=0, right=1, wspace=0, hspace=0)


    ax.set_axis_off()
    fig.add_axes(ax)

    imgs = []
    preds = []
    for i in range(0, 3):
        img_idx = inds[cap_idx, i] * 5
        pred = cap_idx in range(img_idx, img_idx + 5)
        preds.append(pred)

        # restore original indexes
        img_id = int(ld.dataset.ids[img_idx])
        img_info = new_info[img_id]
#         img_info = flickr['images'][img_id]
        img_set = img_info['file_name'].split('_')[1]
        img = Image.open(os.path.join('/opt/jonatas/datasets/lavse/coco/images', img_set, img_info['file_name']))
        crop = transforms.CenterCrop(max(img.size))
        img = crop(img).resize((400,400))
        img = np.asarray(img)
        if i < 2:
            img = np.pad(img, [[0,0], [0,50], [0,0]], mode='constant', constant_values=255)
        imgs.append(img)

    imgs = np.hstack(imgs)
    ax.imshow(imgs)
    plt.text(imgs.shape[1]/2., -30, cap, fontsize=10, horizontalalignment='center')
    first = 200
    for i in range(3):
        symb = '$\checkmark$' if preds[i] else '$\\times$'
        plt.text(first, img.shape[0] + 35, symb, fontsize=15, horizontalalignment='center')
        first += 450
    plt.savefig('/home/jonatas/retrieval_imgs/t2i/{}/t2i_{}_{}.pdf'.format(lang, lang, cap_idx), bbox_inches='tight', dpi=166, pad_inches=0)
    plt.show()
    plt.close()
    

# old code below

---------------------------

In [ ]:
import matplotlib.font_manager
matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

In [ ]:
# success cases
lang = loader.dataset.lang
print('sim matrix shape', sims.shape)
inds = np.argsort(sims, axis=1)[:, ::-1]
print('inds shape', inds.shape)
from matplotlib.transforms import Bbox

max_length = 60
ld = loader
for k in np.random.choice(np.arange(5, 5000, 5), 100, replace=False):#range(5, 5000, 5):
    img_ix = k
    img_id = int(ld.dataset.ids[img_ix])
#     print 'img id', img_id
    img_info = flickr['images'][img_id]
    img = Image.open(os.path.join('/opt/jonatas/datasets/lavse/f30k/images/flickr30k_images', img_info['filename']))
    crop = transforms.CenterCrop(max(img.size))
    img = crop(img).resize((400,400))

    fig = plt.figure(figsize=(4, 4))
#     fig.patch.set_visible(False)
    ax = fig.add_subplot(111)
    plt.axis('off')
    plt.subplots_adjust(bottom=0, top=1, left=0, right=1, wspace=0, hspace=0)
    
    
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(img)
    gt = range(img_ix, img_ix + 5)

    preds = []
    caps = []
    for i in range(5):
        pred = inds[img_ix, i] in gt
        preds.append(pred)
        caps.append(ld.dataset.captions[inds[img_ix, i]])

    n = 8
    big_str = []
    biggest_cap = 0
    for i in range(5):
        cap = caps[i]
        if cap.endswith('.'):
            cap = cap[:-2]
        split_str = cap.split(' ')
        if len(split_str) > n:
            for j in range(n, len(split_str), n):
                split_str[j-1] += '\n   '
        split_str = ' '.join(split_str)
        symb = '$\checkmark$' if preds[i] else '$\\times$'
        br = ''
        if i < 4:
            br = '\n\n'
        
        new_cap = ''.join(split_str) + '.'
        
            
        big_str.append('{}. {} {}{}'.format((i+1), new_cap, symb, br))
    text = ''.join(big_str)
    ax.text(img.width + 30, img.height/2, text, verticalalignment='center', fontsize=15)
    
#     if all(preds):
#         print("all corect found :)")
    plt.savefig('/home/douglas/retrieval_imgs/i2t/random/{}/t2i_{}_{}.pdf'.format(lang, lang, img_ix), bbox_inches=Bbox([[0,0],[10,4]]), dpi=166, pad_inches=0)
    if k % 500 == 0:
        print('processed {} images'.format(k/5))
#     plt.show()
    plt.close()
#     break


In [ ]:
# failure cases

print 'sim matrix shape', sims.shape
inds = np.argsort(sims, axis=1)[:, ::-1]
print 'inds shape', inds.shape
from matplotlib.transforms import Bbox

max_length = 60

ld = loader_de

img_ix = 815
img_id = int(ld.dataset.ids[img_ix])
img_info = flickr['images'][img_id]
img = Image.open(os.path.join('/opt/datasets/flickr/flickr30k_images', img_info['filename']))
crop = transforms.CenterCrop(max(img.size))
img = crop(img).resize((400,400))

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111)
plt.axis('off')
plt.subplots_adjust(bottom=0, top=1, left=0, right=1, wspace=0, hspace=0)


ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(img)
gt = range(img_ix, img_ix + 5)

preds = []
caps = []
for i in range(5):
    pred = inds[img_ix, i] in gt
    preds.append(pred)
    caps.append(ld.dataset.captions[inds[img_ix, i]])

n = 7
big_str = []
biggest_cap = 0
for i in range(5):
    cap = caps[i]
    print cap
    if cap.endswith('.'):
        cap = cap[:-2]
    split_str = cap.split(' ')
    if len(split_str) > n:
        for j in range(n, len(split_str), n):
            split_str[j-1] += '\n   '
    split_str = ' '.join(split_str)
    symb = '$\checkmark$' if preds[i] else '$\\times$'
    br = ''
    if i < 4:
        br = '\n\n'

    new_cap = ''.join(split_str) + '.'
    big_str.append('{}. {} {}{}'.format((i+1), new_cap, symb, br))

ax.text(img.width + 30, img.height/2, unicode(''.join(big_str), 'utf-8'), verticalalignment='center', fontsize=15)

plt.savefig('incorrect_predictions/i2t_de_{}_err.pdf'.format(img_ix), bbox_inches=Bbox([[0,0],[10,4]]), dpi=166, pad_inches=0)
plt.show()
plt.close()


In [ ]:
# text 2 img

sims_t = sims.T
sims_t1k = sims_t[:, range(0, sims_t.shape[1], 5)]
print 'sim1k', sims_t1k.shape
inds = np.argsort(sims_t1k, axis=1)[:, ::-1]

ld = loader_de

cap_idx = 3100
cap = ld.dataset.captions[cap_idx]
print cap

n = 10
if cap.endswith('.'):
    cap = cap[:-2]
split_str = cap.split(' ')
if len(split_str) > n:
    for j in range(n, len(split_str), n):
        split_str[j-1] += '\n   '
cap = ' '.join(split_str)
cap = '{}: {}'.format('Query', cap)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.gca().set_axis_off()
plt.margins(0,0)
plt.gca().xaxis.set_major_locator(plt.NullLocator())
plt.gca().yaxis.set_major_locator(plt.NullLocator())
plt.axis('off')
plt.subplots_adjust(bottom=0, top=1, left=0, right=1, wspace=0, hspace=0)


ax.set_axis_off()
fig.add_axes(ax)

imgs = []
preds = []
for i in range(0, 3):
    img_idx = inds[cap_idx, i] * 5
    pred = cap_idx in range(img_idx, img_idx + 5)
    preds.append(pred)
    
    # restore original indexes
    img_id = int(ld.dataset.ids[img_idx])
    img_info = flickr['images'][img_id]
    img = Image.open(os.path.join('/opt/datasets/flickr/flickr30k_images', img_info['filename']))
    crop = transforms.CenterCrop(max(img.size))
    img = crop(img).resize((400,400))
    img = np.asarray(img)
    if i < 2:
        img = np.pad(img, [[0,0], [0,50], [0,0]], mode='constant', constant_values=255)
    imgs.append(img)
        
imgs = np.hstack(imgs)
ax.imshow(imgs)
plt.text(imgs.shape[1]/2., -30, unicode(cap, 'utf-8'), fontsize=10, horizontalalignment='center')
first = 200
for i in range(3):
    symb = '$\checkmark$' if preds[i] else '$\\times$'
    plt.text(first, img.shape[0] + 35, symb, fontsize=15, horizontalalignment='center')
    first += 450
plt.savefig('t2i_imgs/t2i_de_{}.pdf'.format(cap_idx), bbox_inches='tight', dpi=166, pad_inches=0)
plt.show()
